In [ ]:
!pip install cryptography


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.1 MB 17.4 MB/s 


In [ ]:
#Import dependency
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography import exceptions


###1.1. Generating RSA public key and private key

In [ ]:
# Generates a RSA keypair using the cryptography lib and returns a key tuple (public, private)
  
def keygen():
  
  pv_key = rsa.generate_private_key(public_exponent=65537,key_size=1024)
  pub_key = pv_key.public_key()

  return (pv_key, pub_key) 

### 1.2. Signing a message by private key

In [ ]:
# Signs a message by a RSA private key

def sign(message, private) :

  sig = private.sign(message,padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),hashes.SHA256())
  return sig




###1.3 RSA signature verification by public key

In [ ]:
# Verifies a signature by a RSA public key

def verify(message, public, signature) :
  try:
    public.verify(signature,message,padding.PSS(mgf=padding.MGF1(hashes.SHA256()),salt_length=padding.PSS.MAX_LENGTH),hashes.SHA256())
  except exceptions.InvalidSignature:
    print("Signature verification failed",)
  else:
    print("Signature verified")
  return


Test:

In [ ]:
message = b"Hello world"
privkey, pubkey = keygen()
signature = sign(message,privkey)
verify(message, pubkey, signature)


Signature verified


###2.1. Block mining simulation

In [ ]:
# This function takes a block number and some block data as input and returns a valid nounce (i.e., a nounce that yields a hash output starting with '0000')
def mine (block_num , block_data) :
  data = str(block_num) + str(block_data)

  nounce = 0
  found = 0

  while found == 0:
    hash_input = data + str(nounce)
    hash_input_bytes = bytes(hash_input, 'utf-8')
    digest = hashes.Hash(hashes.SHA256())
    digest.update(hash_input_bytes)
    hash_output = digest.finalize()
    hash_output_hex = hash_output.hex()
    if hash_output_hex[:4] == '0000': 
      found = 1
      break
    nounce += 1


  print(nounce)
  print(hash_output_hex)

  return


  


Test:

In [ ]:
mine(12, "hello")

28592
000072a7327eec0583b858d6541d7fed509d76a2d1be6c9b01a9042370516921


###2.2. Nounce verification

In [ ]:
def verify_nounce (block_num, block_data, nounce):
   data = str(block_num) + str(block_data) + str (nounce)

   hash_input = bytes(data, 'utf-8')
   digest = hashes.Hash(hashes.SHA256())
   digest.update(hash_input) 
   hash_output = digest.finalize()
   hash_output_hex = hash_output.hex()
   if hash_output_hex[:4] == '0000': 
     print("Nounce is valid")
   else:
     print("Nounce is not valid")

   return

Test

In [ ]:

verify_nounce (12, "hello", 28592)
verify_nounce (12, "hello", 28593)


Nounce is valid
Nounce is not valid
